# Example of using the orm classes

Imports

In [1]:
from core.core import Project
from core.orm import Borehole, Interval, Component, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
! rm ./tmp/test_orm_db.db

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-11 16:13:58,361 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-11 16:13:58,362 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 16:13:58,364 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-11 16:13:58,365 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 16:13:58,366 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-11 16:13:58,368 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 16:13:58,373 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-11 16:13:58,374 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 16:13:58,374 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Intervals")
2020-12-11 16:13:58,375 INFO sqlalchemy.engine.base.Engine ()
2020-12-11 16:13:58,375 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Intervals")
2020-12-11 16:13:58,377 INFO sqlalchemy.engine.base.Engine ()
2020-12-11

Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':'../data/test.las', 'F02':'../data/test.las'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(Borehole(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number}})
        interval_number+=1
        int_id += 1
    boreholes[bh_id].intervals_values = d
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}     

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)

2020-12-11 16:13:58,531 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 16:13:58,535 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-11 16:13:58,535 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2020-12-11 16:13:58,543 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-11 16:13:58,545 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 16:13:58,552 INFO sqlalchemy.engine.base.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description) VALUES (?, ?, ?, ?)
2020-12-11 16:13:58,555 INFO sqlalchemy.engine.base.Engine ((0, 'F01', 0, 'Anhydrite'), (1, 'F01', 1, 'Sandstone, grey, vf-f'), (2, 'F01', 2, 'Anhydrite'), (3, 'F01', 3, 'Dolomite'), (4, 'F01', 4, 'Anhydrite'), (5, 'F01', 5, 'Sandstone, grey, vf-f'), (6, 'F01', 6, 'Siltstone, red'), (7, 'F01', 7, 'Dolomite')  ... displaying 10 of 25 total bound parameter sets ...  (23, 'F01', 23, 'Limestone'), (24, 'F01', 24, 'Volcanic'))
2020-12-11 16:13:58,558 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-11 16:13:58,565 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 16:13:58,567 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"


In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.boreholes[1].id='F33'

In [13]:
p.boreholes[1].id

'F33'

In [14]:
p.commit()

2020-12-11 16:13:58,637 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET id=? WHERE "Boreholes".id = ?
2020-12-11 16:13:58,642 INFO sqlalchemy.engine.base.Engine ('F33', 'F02')
2020-12-11 16:13:58,644 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
p.boreholes[0].intervals[0].description

2020-12-11 16:13:58,662 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-11 16:13:58,667 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-11 16:13:58,672 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-11 16:13:58,675 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-11 16:13:58,676 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [16]:
session.close()

2020-12-11 16:13:58,685 INFO sqlalchemy.engine.base.Engine ROLLBACK
